## Nick Curci
### Homework

#### Due Date: Thursday 4/16 at the start of the class.

- Please try to improve areaUnderROC value for this deposit prediction model. Please document what changes you have made, what is the best model/algorithms you found and new areaUnderROC value.

- please export the completed notebook as HTML, zip it and submit it on Blackboard by due date.

##Predict whether a bank customer will subscribe to a term deposit
The data is related to direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict whether the client will subscribe (Yes/No) to a term deposit. The dataset can be downloaded from [Kaggle](http://www.kaggle.com/rouseguy/bankbalanced).

[Attribute Information](http://archive.ics.uci.edu/ml/datasets/bank+marketing)

Input variables:
##### bank client data:
1. - age (numeric)
2. - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3. - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4. - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5. - default: has credit in default? (categorical: 'no','yes','unknown')
6. - housing: has housing loan? (categorical: 'no','yes','unknown')
7. - loan: has personal loan? (categorical: 'no','yes','unknown')
#### related with the last contact of the current campaign:
8. - contact: contact communication type (categorical: 'cellular','telephone')
9. - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10. - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11. - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
#### other attributes:
12. - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13. - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14. - previous: number of contacts performed before this campaign and for this client (numeric)
15. - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

In [0]:
%fs ls /mnt/classdata/bank

path,name,size
dbfs:/mnt/classdata/bank/bank.csv,bank.csv,918960


In [0]:
#read bank data
bank = spark.read.csv('/mnt/classdata/bank/bank.csv', header = True, inferSchema = True)
bank.printSchema()

root
-- age: integer (nullable = true)
-- job: string (nullable = true)
-- marital: string (nullable = true)
-- education: string (nullable = true)
-- default: string (nullable = true)
-- balance: integer (nullable = true)
-- housing: string (nullable = true)
-- loan: string (nullable = true)
-- contact: string (nullable = true)
-- day: integer (nullable = true)
-- month: string (nullable = true)
-- duration: integer (nullable = true)
-- campaign: integer (nullable = true)
-- pdays: integer (nullable = true)
-- previous: integer (nullable = true)
-- poutcome: string (nullable = true)
-- deposit: string (nullable = true)

In [0]:
display(bank)

age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
42,management,single,tertiary,no,0,yes,yes,unknown,5,may,562,2,-1,0,unknown,yes
56,management,married,tertiary,no,830,yes,yes,unknown,6,may,1201,1,-1,0,unknown,yes
60,retired,divorced,secondary,no,545,yes,no,unknown,6,may,1030,1,-1,0,unknown,yes
37,technician,married,secondary,no,1,yes,no,unknown,6,may,608,1,-1,0,unknown,yes
28,services,single,secondary,no,5090,yes,no,unknown,6,may,1297,3,-1,0,unknown,yes


In [0]:
bank.describe('age', 'balance','duration', 'pdays').show()

+-------+------------------+------------------+------------------+------------------+
summary| age| balance| duration| pdays|
+-------+------------------+------------------+------------------+------------------+
 count| 11162| 11162| 11162| 11162|
 mean|41.231947679627304|1528.5385235620856|371.99381831213043| 51.33040673714388|
 stddev|11.913369192215518| 3225.413325946149|347.12838571630687|108.75828197197717|
 min| 18| -6847| 2| -1|
 max| 95| 81204| 3881| 854|
+-------+------------------+------------------+------------------+------------------+

###Feature Engineering with Transformers

[RFormula](https://spark.apache.org/docs/latest/ml-features#rformula)

In [0]:
from pyspark.ml.feature import RFormula

bank_rf = RFormula(formula="deposit ~ .")

####Prepare (Train and transform) our data frame

In [0]:
fittedRF = bank_rf.fit(bank)

preparedDF = fittedRF.transform(bank)

preparedDF.select('deposit', 'label', 'features').show(10, False)


+-------+-----+---------------------------------------------------------------------------------------------------------------------+
deposit|label|features |
+-------+-----+---------------------------------------------------------------------------------------------------------------------+
yes |1.0 |(42,[0,4,12,14,17,18,20,22,23,24,35,36,37,39],[59.0,1.0,1.0,1.0,1.0,2343.0,1.0,1.0,5.0,1.0,1042.0,1.0,-1.0,1.0]) |
yes |1.0 |(42,[0,4,12,14,17,18,19,20,22,23,24,35,36,37,39],[56.0,1.0,1.0,1.0,1.0,45.0,1.0,1.0,1.0,5.0,1.0,1467.0,1.0,-1.0,1.0])|
yes |1.0 |(42,[0,3,12,14,17,18,20,22,23,24,35,36,37,39],[41.0,1.0,1.0,1.0,1.0,1270.0,1.0,1.0,5.0,1.0,1389.0,1.0,-1.0,1.0]) |
yes |1.0 |(42,[0,5,12,14,17,18,20,22,23,24,35,36,37,39],[55.0,1.0,1.0,1.0,1.0,2476.0,1.0,1.0,5.0,1.0,579.0,1.0,-1.0,1.0]) |
yes |1.0 |(42,[0,4,12,15,17,18,19,20,22,23,24,35,36,37,39],[54.0,1.0,1.0,1.0,1.0,184.0,1.0,1.0,1.0,5.0,1.0,673.0,2.0,-1.0,1.0])|
yes |1.0 |(42,[0,1,13,15,17,22,23,24,35,36,37,39],[42.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,562.0,2.0,-1.0,1.0]) |
yes |1.0 |(42,[0,1,12,15,17,18,22,23,24,35,36,37,39],[56.0,1.0,1.0,1.0,1.0,830.0,1.0,6.0,1.0,1201.0,1.0,-1.0,1.0]) |
yes |1.0 |(42,[0,6,14,17,18,20,22,23,24,35,36,37,39],[60.0,1.0,1.0,1.0,545.0,1.0,1.0,6.0,1.0,1030.0,1.0,-1.0,1.0]) |
yes |1.0 |(42,[0,3,12,14,17,18,20,22,23,24,35,36,37,39],[37.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6.0,1.0,608.0,1.0,-1.0,1.0]) |
yes |1.0 |(42,[0,5,13,14,17,18,20,22,23,24,35,36,37,39],[28.0,1.0,1.0,1.0,1.0,5090.0,1.0,1.0,6.0,1.0,1297.0,3.0,-1.0,1.0]) |
+-------+-----+---------------------------------------------------------------------------------------------------------------------+
only showing top 10 rows

split our data into train and test set

# FIRST CHANGE, CHANGED THE SPLITS TO 90-10

In [0]:
(train, test) = preparedDF.randomSplit([0.9, 0.1], seed=100)


In [0]:
train.count()
test.count()

Out[8]: 1137

####Instantiate an instance of LogisticsRegression, set the label columns and the feature columns

In [0]:
from pyspark.ml.classification import LogisticRegression
#lr = LogisticRegression(labelCol="label",featuresCol="features")
lr= LogisticRegression()

### Train the logistics regression model based on training dataset

In [0]:
lrModel = lr.fit(train)

###Using the model making prediction

In [0]:
lrPrediction=lrModel.transform(test)
lrPrediction.select("label", "prediction").show(10, False)

+-----+----------+
label|prediction|
+-----+----------+
1.0 |1.0 |
1.0 |0.0 |
1.0 |1.0 |
0.0 |1.0 |
1.0 |1.0 |
0.0 |0.0 |
1.0 |1.0 |
1.0 |0.0 |
1.0 |1.0 |
1.0 |1.0 |
+-----+----------+
only showing top 10 rows

calcualte accuracy

In [0]:
print("prediction accuracy is: ", lrPrediction.where("prediction==label").count()/lrPrediction.count())

tp=lrPrediction.where("label=1 and prediction=1").count()
fp=lrPrediction.where("label=0 and prediction=1").count()
tn=lrPrediction.where("label=0 and prediction=0").count()
fn=lrPrediction.where("label=1 and prediction=0").count()

print("true positive is: ", tp)

print("false positive is: ", fp)

print("true negative is: ", tn)

print("false negative is ", fn)

print("precision is ", tp/(tp+fp)) 

print("recall is ", tp/(tp+fn))


prediction accuracy is: 0.8337730870712401
true positive is: 425
false positive is: 89
true negative is: 523
false negative is 100
precision is 0.8268482490272373
recall is 0.8095238095238095

In [0]:
featureIndex=preparedDF.schema["features"].metadata["ml_attr"]["attrs"]
x=0
#print numberic feature
for x in range(len(lrModel.coefficients)-1):
  try:
    print("feature", featureIndex["numeric"][x]['idx'], " ", featureIndex["numeric"][x]['name'], ': ', lrModel.coefficients[x])
  except:
    continue

# print binary feature   
for x in range(len(lrModel.coefficients)-1):
  try:
    print("feature", featureIndex["binary"][x]['idx'], " ", featureIndex["binary"][x]['name'], ': ', lrModel.coefficients[x])
  except:
    continue

feature 0 age : -0.0003925271277357672
feature 18 balance : 0.3989315461408882
feature 23 day : 0.37618195364897994
feature 35 duration : 0.5101451892657288
feature 36 campaign : 0.6838019867101518
feature 37 pdays : 0.388890360825208
feature 38 previous : 0.9228368994063423
feature 1 job_management : -0.0003925271277357672
feature 2 job_blue-collar : 0.3989315461408882
feature 3 job_technician : 0.37618195364897994
feature 4 job_admin. : 0.5101451892657288
feature 5 job_services : 0.6838019867101518
feature 6 job_retired : 0.388890360825208
feature 7 job_self-employed : 0.9228368994063423
feature 8 job_student : 0.16980311535165477
feature 9 job_unemployed : 1.2875954417449877
feature 10 job_entrepreneur : 0.6158336212704281
feature 11 job_housemaid : 0.1817337833667115
feature 12 marital_married : 0.1785929356186164
feature 13 marital_single : -0.20564570495882248
feature 14 education_secondary : 0.07172965151593574
feature 15 education_tertiary : -0.03679657090801749
feature 16 education_primary : 0.23966038278260365
feature 17 default_no : -0.27428524165776097
feature 19 housing_no : -0.02664907665452865
feature 20 loan_no : 3.674444116681561e-05
feature 21 contact_cellular : 0.6877878420450214
feature 22 contact_unknown : 0.4392850447678706
feature 24 month_may : 0.03329897899029602
feature 25 month_aug : -1.5729755529113714
feature 26 month_jul : 0.004655433094409426
feature 27 month_jun : -2.0665319110209293
feature 28 month_nov : -2.202815586137976
feature 29 month_apr : -2.4322080097037366
feature 30 month_feb : -1.1222998185419475
feature 31 month_oct : -2.3333999512124097
feature 32 month_jan : -1.3926167011287056
feature 33 month_sep : -1.5900606652839242
feature 34 month_mar : -0.34495805661171824
feature 39 poutcome_unknown : -2.6665118272396224
feature 40 poutcome_failure : -0.45917337012783305
feature 41 poutcome_success : 0.7353869615401302

Evalaute model

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()\
  .setMetricName("areaUnderROC")\
  .setRawPredictionCol("prediction")\
  .setLabelCol("label")

# NEW PREDICTION VALUE

In [0]:
evaluator.evaluate(lrPrediction)

Out[15]: 0.8320494864612511

In [0]:
# ROC for test data
display(lrModel, test, "ROC")

False Positive Rate,True Positive Rate,Threshold
0.0,0.0,0.9993367043299846
0.0,0.022222222222222223,0.9993367043299846
0.0,0.044444444444444446,0.9940096868137981
0.0,0.06666666666666667,0.9890170455012353
0.0,0.08888888888888889,0.9886112695254684
0.0,0.1111111111111111,0.988292869056298
0.0,0.13333333333333333,0.9866091853935158
0.0,0.15555555555555556,0.9841169044607524
0.0,0.17777777777777778,0.9838073646782458
0.0,0.2,0.9522294441090031


In [0]:
display(lrModel.summary.roc)

FPR,TPR
0.0,0.0
0.0013305455236647026,0.01952141057934509
0.0019007793195210036,0.039882451721242655
0.003231324843185706,0.059403862300587744
0.004942026230754609,0.0785054575986566
0.006462649686371412,0.09781696053736356
0.007983273141988215,0.11712846347607053
0.009693974529557118,0.13623005877413938
0.011404675917126022,0.15533165407220823
0.012735221440790725,0.1748530646515533


In [0]:
display(lrModel.summary.pr)

recall,precision
0.0,0.93
0.01952141057934509,0.93
0.039882451721242655,0.95
0.059403862300587744,0.9433333333333334
0.0785054575986566,0.935
0.09781696053736356,0.932
0.11712846347607053,0.93
0.13623005877413938,0.9271428571428572
0.15533165407220823,0.925
0.1748530646515533,0.9255555555555556


## Decision Trees

You can read more about [Decision Trees](http://spark.apache.org/docs/latest/mllib-decision-tree.html) in the Spark MLLib Programming Guide.
The Decision Trees algorithm is popular because it handles categorical
data and works out of the box with multiclass classification tasks.

# SECOND CHANGE, USED MAX DEPTH OF 6 FOR THE DECISION TREE

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=6)

# Train model with Training Data
dtModel = dt.fit(train)

In [0]:
display(dtModel)

treeNode
"{""index"":49,""featureType"":""continuous"",""prediction"":null,""threshold"":205.5,""categories"":null,""feature"":35,""overflow"":false}"
"{""index"":21,""featureType"":""categorical"",""prediction"":null,""threshold"":null,""categories"":[1.0],""feature"":41,""overflow"":false}"
"{""index"":11,""featureType"":""continuous"",""prediction"":null,""threshold"":125.5,""categories"":null,""feature"":35,""overflow"":false}"
"{""index"":5,""featureType"":""continuous"",""prediction"":null,""threshold"":73.5,""categories"":null,""feature"":35,""overflow"":false}"
"{""index"":1,""featureType"":""continuous"",""prediction"":null,""threshold"":182.5,""categories"":null,""feature"":37,""overflow"":false}"
"{""index"":0,""featureType"":null,""prediction"":0.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":3,""featureType"":""continuous"",""prediction"":null,""threshold"":57.5,""categories"":null,""feature"":35,""overflow"":false}"
"{""index"":2,""featureType"":null,""prediction"":0.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":4,""featureType"":null,""prediction"":1.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":9,""featureType"":""continuous"",""prediction"":null,""threshold"":6.5,""categories"":null,""feature"":38,""overflow"":false}"


In [0]:
featureIndex=preparedDF.schema["features"].metadata["ml_attr"]["attrs"]

print(featureIndex)

{'numeric': [{'idx': 0, 'name': 'age'}, {'idx': 18, 'name': 'balance'}, {'idx': 23, 'name': 'day'}, {'idx': 35, 'name': 'duration'}, {'idx': 36, 'name': 'campaign'}, {'idx': 37, 'name': 'pdays'}, {'idx': 38, 'name': 'previous'}], 'binary': [{'idx': 1, 'name': 'job_management'}, {'idx': 2, 'name': 'job_blue-collar'}, {'idx': 3, 'name': 'job_technician'}, {'idx': 4, 'name': 'job_admin.'}, {'idx': 5, 'name': 'job_services'}, {'idx': 6, 'name': 'job_retired'}, {'idx': 7, 'name': 'job_self-employed'}, {'idx': 8, 'name': 'job_student'}, {'idx': 9, 'name': 'job_unemployed'}, {'idx': 10, 'name': 'job_entrepreneur'}, {'idx': 11, 'name': 'job_housemaid'}, {'idx': 12, 'name': 'marital_married'}, {'idx': 13, 'name': 'marital_single'}, {'idx': 14, 'name': 'education_secondary'}, {'idx': 15, 'name': 'education_tertiary'}, {'idx': 16, 'name': 'education_primary'}, {'idx': 17, 'name': 'default_no'}, {'idx': 19, 'name': 'housing_no'}, {'idx': 20, 'name': 'loan_no'}, {'idx': 21, 'name': 'contact_cellular'}, {'idx': 22, 'name': 'contact_unknown'}, {'idx': 24, 'name': 'month_may'}, {'idx': 25, 'name': 'month_aug'}, {'idx': 26, 'name': 'month_jul'}, {'idx': 27, 'name': 'month_jun'}, {'idx': 28, 'name': 'month_nov'}, {'idx': 29, 'name': 'month_apr'}, {'idx': 30, 'name': 'month_feb'}, {'idx': 31, 'name': 'month_oct'}, {'idx': 32, 'name': 'month_jan'}, {'idx': 33, 'name': 'month_sep'}, {'idx': 34, 'name': 'month_mar'}, {'idx': 39, 'name': 'poutcome_unknown'}, {'idx': 40, 'name': 'poutcome_failure'}, {'idx': 41, 'name': 'poutcome_success'}]}

In [0]:
# Make predictions on test data using the Transformer.transform() method.
dtPrediction = dtModel.transform(test)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(dtPrediction)

Out[23]: 0.7956146903205727

## Random Forest

Random Forests uses an ensemble of trees to improve model accuracy.
You can read more about [Random Forest] from the [classification and regression] section of MLlib Programming Guide.

[classification and regression]: https://spark.apache.org/docs/latest/ml-classification-regression.html
[Random Forest]: https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forests

In [0]:
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Train model with Training Data
rfModel = rf.fit(train)

In [0]:
# Make predictions on test data using the Transformer.transform() method.
rfPrediction = rfModel.transform(test)

We will evaluate our Random Forest model with BinaryClassificationEvaluator.

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(rfPrediction)

Out[26]: 0.8688406473700597

In [0]:
print("prediction accuracy is: ", rfPrediction.where("prediction==label").count()/rfPrediction.count())

tp=lrPrediction.where("label=1 and prediction=1").count()
fp=lrPrediction.where("label=0 and prediction=1").count()
tn=lrPrediction.where("label=0 and prediction=0").count()
fn=lrPrediction.where("label=1 and prediction=0").count()

print("true positive is: ", tp)

print("false positive is: ", fp)

print("true negative is: ", tn)

print("false negative is ", fn)

print("precision is ", tp/(tp+fp)) 

print("recall is ", tp/(tp+fn))


prediction accuracy is: 0.7739665787159191
true positive is: 425
false positive is: 89
true negative is: 523
false negative is 100
precision is 0.8268482490272373
recall is 0.8095238095238095

### Gradient-Boosted Trees

In [0]:
from pyspark.ml.classification import GBTClassifier
gbtClassifier=GBTClassifier()

gbtModel=gbtClassifier.fit(train)

# Make predictions on test data using the Transformer.transform() method.
gbtPrediction = gbtModel.transform(test)

# Evaluate Model
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(gbtPrediction)

Out[28]: 0.9089060068471873

In [0]:
print("prediction accuracy is: ", gbtPrediction.where("prediction==label").count()/gbtPrediction.count())

tp=lrPrediction.where("label=1 and prediction=1").count()
fp=lrPrediction.where("label=0 and prediction=1").count()
tn=lrPrediction.where("label=0 and prediction=0").count()
fn=lrPrediction.where("label=1 and prediction=0").count()

print("true positive is: ", tp)

print("false positive is: ", fp)

print("true negative is: ", tn)

print("false negative is ", fn)

print("precision is ", tp/(tp+fp))

print("recall is ", tp/(tp+fn))

prediction accuracy is: 0.8478452066842568
true positive is: 425
false positive is: 89
true negative is: 523
false negative is 100
precision is 0.8268482490272373
recall is 0.8095238095238095

###Make Predictions

As Gradient-boosted Trees model gives us the best areaUnderROC value, we will use the bestModel obtained from Gradient-Boosted Tree model for deployment, and use it to generate predictions on new data. In this example, we will simulate this by generating predictions on the entire dataset.

In [0]:
# Generate predictions for entire dataset
finalPredictions = gbtModel.transform(preparedDF)

In [0]:
# Evaluate best model
evaluator.evaluate(finalPredictions)

Out[31]: 0.9321993476528805

In [0]:
# create a SQL view
finalPredictions.createOrReplaceTempView("finalPredictions")

In an operational environment, analysts may use a similar machine learning pipeline to obtain predictions on new data, organize it into a table and use it for analysis or lead targeting.

In [0]:
%sql
select case when label=1 and prediction=1 then "true positive"
            when label=1 and prediction=0 then "false negative"
            when label=0 and prediction=0 then "true negative"
            when label=0 and prediction=1 then "false positive"
            else "N/A"
            End as status, count(*) as NumberofRecords
from finalPredictions 
group by status

status,NumberofRecords
true positive,4664
true negative,4938
false negative,625
false positive,935


# END
## The training and testing datasets were modified to a 9:1 ratio, the ROC value increased by over .1, the decision tree was changed to allow a max depth of 6 rather than the original 3 and this increase gave us a better model. I think that the decision tree classifier is the best of the classifiers because it allows for direct manipulation of the depth of search